<a href="https://colab.research.google.com/github/etziosplay4u/Basics-of-AI-in-ML-DL/blob/main/BasicANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
import torchvision

# Basic Prep
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2), (0.3))])

# Load
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform))

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transform))

100%|██████████| 9.91M/9.91M [00:00<00:00, 12.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 338kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.20MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.91MB/s]


In [2]:
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, epochs=20):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 10)

        self.epochs = epochs

    def forward_pass(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)
        x = torch.sigmoid(x)
        x = self.linear3(x)
        x = torch.softmax(x, dim=0)
        return x

    def one_hot_encode(self, y):
        encoded = torch.zeros([10], dtype=torch.float64)
        encoded[y[0]] = 1.
        return encoded

    def train_model(self, train_loader, optimizer, criterion):
        start_time = time.time()
        loss = None

        for iteration in range(self.epochs):
            for x,y in train_loader:
                y = self.one_hot_encode(y)
                optimizer.zero_grad()
                output = self.forward_pass(torch.flatten(x))
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()

            print('Epoch: {0}, Time Spent: {1:.2f}s, Loss: {2}'.format(
                iteration+1, time.time() - start_time, loss
            ))

    def predict(self, output_img):
      self.eval()
      with torch.no_grad():
          output = self.forward_pass(torch.flatten(output_img))
          predicted_class = torch.argmax(output)
      return predicted_class

In [ ]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

model.train_model(train_loader, optimizer, criterion)

Epoch: 1, Time Spent: 101.19s, Loss: 0.7351984851062298
Epoch: 2, Time Spent: 199.73s, Loss: 0.7381450887769461
Epoch: 3, Time Spent: 297.85s, Loss: 0.7407056482508778


# Output

In [ ]:
from PIL import Image

image = Image.open('/content/images.jpeg').convert('L')

output_img = transform(image)

# Predict
predicted_class = model.predict(output_img)
print(f'Predicted Class: {predicted_class}')